<img src="images/inmas.png" width=130x align='right' />

# Exercises 23 - OpenAI Application Programming Interface
    
This notebook requires that you concurrently use a terminal. Use virtual desktops if possible (e.g., Windows Key+Ctl+arrows).

### Prerequisite
Notebook 23

In [ ]:
import wave, struct, os
from openai import OpenAI                        # The API to OpenAI
from pvrecorder import PvRecorder                # To record our voice
from playsound import playsound                  # To play the generated speech
from IPython.display import Image, display       # To show generated images

# To silence deprecation warnings
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

### 1.
Experiment with different voices and cues for the personality of the system and for the image generation in the last version of the ChatBot class.

### 2.
Use a call to 'gpt-4o' to generate code in python to play Hangman. Copy the response message to a cell and run the program to see if it behaves as it should. You will need an API key and some credits on OpenAI to accomplish this task.

### 3.*
This exercise further develops the class ChatBot that was presented in the Notebook. We reproduce it here for your convenience:

In [ ]:
class ChatBot:
    '''A class to interact with OpenAI API and keep track of context.'''
    def __init__(self, client, model):
        self.client = client
        self.model = model
        self.context = [{'role': 'system', 'content': 'You are math professor with a dark sense of humor.'}]

    def chat(self, question):
        self.context.append({'role': 'user', 'content': question})
        response = self.client.chat.completions.create(model=self.model, messages=self.context)
        response_content = response.choices[0].message.content
        self.context.append({'role': 'assistant', 'content': response_content})
        self.print_chat()
        self.speak(response_content, len(self.context)/2)

    def speak(self, message, index=0):
        speech_file = os.getcwd() + '\\_speech_%03d.mp3'%index
        response = client.audio.speech.create(model='tts-1-hd', voice='echo', input=message)
        if os.path.exists(speech_file):
            os.remove(speech_file)
        response.stream_to_file(speech_file)
        playsound(speech_file)
        
    def print_chat(self):
        for message in self.context:
            if message['role'] == 'user':
                print('USER: %s' % message['content'])
            elif message['role'] == 'assistant':
                print('BOT: %s' % message['content'])
    

We would like to speak to our Bot and convert our speech to text using the *Whisper* model in OpenAI. 

The first step is to record our voice using the following code. It records until a kernel interrupt is sent. 

In [ ]:
import time

index = 0
audiofile = '_audio_recording%03d.wav'%index
recorder = PvRecorder(device_index=-1, frame_length=512)
audio = []

recorder.start()
print('Audio recording started ... you have 10 seconds:')
t_end = time.time() + 10
while time.time() < t_end:
        frame = recorder.read()
        audio.extend(frame)
finally:
        recorder.delete()

recorder.stop()
print('Audio recording stopped ...')
with wave.open(audiofile, 'w') as f:
    f.setparams((1, 2, 1600, 512, 'NONE', 'NONE'))
    f.writeframes(struct.pack('h'*len(audio), *audio))

We now use the audiofile which contains our recorded voice and pass it to *Whisper* for transcription:

In [ ]:
with open(audiofile) as f:
    transcript = client.audio.transcription.create(model='whisper-1', file=f)

transcript.text

Using the content of the last cell for a method called `transcribe(self, audiofile)` which returns the transcribed text to be passed as a message, and the content of the previous one in a method called `record_audio(self, index=0)` which return the path of the audio file,  integrate the last two cells in the ChatBot class. One additional method should be added:

In [ ]:
def voicechat(self):
    recorded_audio = self.record_audio(index=len(self.context)/2)   # We use the response count as a numbered unique filename
    message = self.transcribe(recorded_audio)
    self.chat(message)